## Parte 1: Carregamento e Limpeza de Dados

Para começar, carregamos nossa principal ferramenta de trabalho, a biblioteca pandas, e em seguida, importamos as 4 planilhas de dados (partidas, estatísticas, cartões e gols).

# 📊 Análise de Dados do Brasileirão

Bem-vindo(a) ao notebook de análise do Campeonato Brasileiro! Aqui você encontrará:

- **Carregamento e Limpeza de Dados:** Importação dos dados brutos, tratamento de valores ausentes e padronização das tabelas.
- **Construção da Tabela Mestra:** Unificação das informações de partidas, estatísticas, cartões e gols em uma única base limpa e confiável.
- **Análises Estatísticas:** Exploração dos fatores que influenciam o resultado das partidas, como posse de bola, chutes, mandante/visitante, entre outros.
- **Visualizações Interativas:** Gráficos e tabelas para facilitar a interpretação dos dados e gerar insights para a equipe técnica.

---

> **Objetivo:** Descobrir padrões e fatores de sucesso no futebol brasileiro, respondendo perguntas estratégicas e auxiliando na tomada de decisão.

---

## ⚙️ Estrutura do Notebook

1. **Parte 1:** Carregamento e Limpeza de Dados
2. **Parte 2:** Construção da Tabela Mestra
3. **Parte 3:** Análises e Visualizações

---

Vamos começar! 🚀

## ✨ **PARTE 1 - Carregamento e Limpeza de Dados** 🧹

In [152]:
import pandas as pd

In [153]:
try:
    df_full = pd.read_csv('Brasileirao_Dataset/campeonato-brasileiro-full.csv')
    df_stats = pd.read_csv('Brasileirao_Dataset/campeonato-brasileiro-estatisticas-full.csv')
    df_cartoes = pd.read_csv('Brasileirao_Dataset/campeonato-brasileiro-cartoes.csv')
    df_gols = pd.read_csv('Brasileirao_Dataset/campeonato-brasileiro-gols.csv')

    print("Os 4 arquivos CSV foram carregados com sucesso!")
    
except FileNotFoundError as e:
    print(f"Erro: O arquivo {e.filename} não foi encontrado.")
    print("Por favor, certifique-se de que os arquivos CSV estão no mesmo diretório que o seu notebook.")

Os 4 arquivos CSV foram carregados com sucesso!


In [154]:
# Filtro de Qualidade em df_stats

print(f"Número de linhas em df_stats ANTES do filtro: {len(df_stats)}")

# Lista de colunas de estatísticas numéricas que indicam atividade em um jogo
colunas_de_atividade = ['chutes', 'faltas', 'cartao_amarelo', 'cartao_vermelho', 'impedimentos', 'escanteios']

# Calculamos a soma das estatísticas para cada linha. Se a soma for 0, significa que não há dados.
soma_stats = df_stats[colunas_de_atividade].sum(axis=1)

# Mantemos apenas as linhas onde a soma é maior que 0
df_stats_com_dados = df_stats[soma_stats > 0].copy()

print(f"Número de linhas em df_stats DEPOIS do filtro: {len(df_stats_com_dados)}")
print("\nFiltro de qualidade aplicado com sucesso! Apenas partidas com dados de estatísticas foram mantidas.")

Número de linhas em df_stats ANTES do filtro: 17570
Número de linhas em df_stats DEPOIS do filtro: 6820

Filtro de qualidade aplicado com sucesso! Apenas partidas com dados de estatísticas foram mantidas.


### ⏰ Ajuste de Data, Hora e Colunas

- **Datas e horários** convertidos para análise temporal.
- **Coluna `ID`** padronizada para `partida_id`.

In [155]:
# Ajustando os tipos de dados de data e hora
df_full['data'] = pd.to_datetime(df_full['data'], format='%d/%m/%Y', errors='coerce')
df_full['hora'] = pd.to_datetime(df_full['hora'], format='%H:%M', errors='coerce').dt.time

# Renomeando a coluna 'ID' para 'partida_id' para padronização
df_full.rename(columns={'ID': 'partida_id'}, inplace=True)

print("Formatos de data/hora e coluna de ID ajustados com sucesso.")

Formatos de data/hora e coluna de ID ajustados com sucesso.


#### Sincronização das Tabelas

Só seguimos com partidas que têm dados completos em todas as tabelas — máxima qualidade para as análises!

In [156]:
# Encontrando os IDs de partida comuns a todas as tabelas
ids_full = set(df_full['partida_id'])
ids_stats = set(df_stats_com_dados['partida_id'])
ids_cartoes = set(df_cartoes['partida_id'])
ids_gols = set(df_gols['partida_id'])

# Encontra a interseção de IDs
common_ids = set.intersection(ids_full, ids_stats, ids_cartoes, ids_gols)

print(f"Número de partidas em comum a todas as 4 bases: {len(common_ids)}")

Número de partidas em comum a todas as 4 bases: 3059


Agora, filtramos as tabelas para manter apenas as partidas validadas em todas as bases.  
🔎 Apenas jogos com dados completos seguem para as próximas análises!

In [157]:
# Filtrando os DataFrames para manter apenas os dados completos
common_ids_list = list(common_ids)

df_full_cleaned = df_full[df_full['partida_id'].isin(common_ids_list)].copy()
df_stats_cleaned = df_stats_com_dados[df_stats_com_dados['partida_id'].isin(common_ids_list)].copy()
df_cartoes_cleaned = df_cartoes[df_cartoes['partida_id'].isin(common_ids_list)].copy()
df_gols_cleaned = df_gols[df_gols['partida_id'].isin(common_ids_list)].copy()

print("Filtro aplicado. Comparativo do número de linhas (Antes -> Depois):")
print(f"Partidas:      {len(df_full)} -> {len(df_full_cleaned)}")
print(f"Estatísticas:  {len(df_stats_com_dados)} -> {len(df_stats_cleaned)}")
print(f"Cartões:       {len(df_cartoes)} -> {len(df_cartoes_cleaned)}")
print(f"Gols:          {len(df_gols)} -> {len(df_gols_cleaned)}")

Filtro aplicado. Comparativo do número de linhas (Antes -> Depois):
Partidas:      8785 -> 3059
Estatísticas:  6820 -> 6118
Cartões:       20953 -> 15659
Gols:          9861 -> 7928


#### Tratando Campos Vazios

Preenchimento inteligente dos campos ausentes para evitar erros e inconsistências.  
- **Formações dos times:** colunas removidas por baixa relevância nesta análise.
- **Campos como 'posse de bola':** mantidos como NaN para análises futuras.

In [158]:
# Removendo as colunas de formação do DataFrame df_full_cleaned
df_full_cleaned = df_full_cleaned.drop(columns=['formacao_mandante', 'formacao_visitante'], errors='ignore')

# Em df_stats, estatísticas ausentes viram 'Sem Info'
df_stats_cleaned = df_stats_cleaned.copy()
df_stats_cleaned['posse_de_bola'] = df_stats_cleaned['posse_de_bola'].fillna('Sem Info')
df_stats_cleaned['precisao_passes'] = df_stats_cleaned['precisao_passes'].fillna('Sem Info')

# Em df_cartoes, removemos a coluna 'num_camisa' e preenchemos 'posicao'
df_cartoes_cleaned = df_cartoes_cleaned.drop(columns=['num_camisa'], errors='ignore')
df_cartoes_cleaned = df_cartoes_cleaned.copy()
df_cartoes_cleaned['posicao'] = df_cartoes_cleaned['posicao'].fillna('Sem Info')

# Em df_gols, um gol sem tipo definido é assumido como 'Gol Normal'
df_gols_cleaned = df_gols_cleaned.copy()
df_gols_cleaned['tipo_de_gol'] = df_gols_cleaned['tipo_de_gol'].fillna('Gol Normal')

print("Campos vazios tratados com sucesso.")


Campos vazios tratados com sucesso.


## 🏗️ Parte 2: Construindo a Tabela Mestra 

Com os dados limpos e sincronizados, o objetivo agora é criar uma única tabela onde cada linha representa uma partida e contém as estatísticas do time da casa e do visitante, lado a lado.

---

### 🆚 Separando Estatísticas: Mandante x Visitante

Primeiro, separamos os dados da tabela de estatísticas em duas novas tabelas:
- 🏠 **Mandantes** (times da casa)
- 🚗 **Visitantes**

Assim, cada partida terá as informações dos dois lados, prontas para análise!

In [159]:
# Separando as estatísticas
stats_com_times = pd.merge(df_stats_cleaned, df_full_cleaned[['partida_id', 'mandante', 'visitante']], on='partida_id')
mandante_stats = stats_com_times[stats_com_times['clube'] == stats_com_times['mandante']].copy()
visitante_stats = stats_com_times[stats_com_times['clube'] == stats_com_times['visitante']].copy()

print("Estatísticas separadas para mandantes e visitantes.")

Estatísticas separadas para mandantes e visitantes.


In [160]:
mandante_stats.head(3)

,partida_id,rodata,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,cartao_vermelho,impedimentos,escanteios,mandante,visitante
0,4741,13,Chapecoense,15,0,35%,280,Sem Info,19,4,0,0,8,Chapecoense,Flamengo
3,4808,20,Palmeiras,10,0,44%,384,Sem Info,18,4,0,1,5,Palmeiras,Criciuma
5,4833,22,Coritiba,9,0,43%,360,Sem Info,14,5,0,1,11,Coritiba,Sao Paulo


In [161]:
visitante_stats.head(3)

,partida_id,rodata,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,cartao_vermelho,impedimentos,escanteios,mandante,visitante
1,4741,13,Flamengo,9,0,65%,509,Sem Info,13,1,0,1,5,Chapecoense,Flamengo
2,4808,20,Criciuma,10,0,56%,471,Sem Info,17,3,0,1,6,Palmeiras,Criciuma
4,4833,22,Sao Paulo,5,0,57%,481,Sem Info,10,1,0,3,2,Coritiba,Sao Paulo


Para identificar facilmente a qual time cada estatística pertence na tabela final, adicionamos os prefixos **`mandante_`** e **`visitante_`** às colunas correspondentes.

In [162]:
# Adicionando prefixos
colunas_stats = [
    'chutes', 'chutes_no_alvo', 'posse_de_bola', 'passes', 'precisao_passes',
    'faltas', 'cartao_amarelo', 'cartao_vermelho', 'impedimentos', 'escanteios'
]

# Prefixando e selecionando colunas para a tabela de mandantes
mandante_stats_renamed = mandante_stats[colunas_stats].add_prefix('mandante_')
mandante_stats_renamed['partida_id'] = mandante_stats['partida_id']

# Prefixando e selecionando colunas para a tabela de visitantes
visitante_stats_renamed = visitante_stats[colunas_stats].add_prefix('visitante_')
visitante_stats_renamed['partida_id'] = visitante_stats['partida_id']

print("Prefixos adicionados com sucesso!")
display(mandante_stats_renamed.head(300))

Prefixos adicionados com sucesso!


,mandante_chutes,mandante_chutes_no_alvo,mandante_posse_de_bola,mandante_passes,mandante_precisao_passes,mandante_faltas,mandante_cartao_amarelo,mandante_cartao_vermelho,mandante_impedimentos,mandante_escanteios,partida_id
0,15,0,35%,280,Sem Info,19,4,0,0,8,4741
3,10,0,44%,384,Sem Info,18,4,0,1,5,4808
5,9,0,43%,360,Sem Info,14,5,0,1,11,4833
7,15,0,42%,357,Sem Info,20,2,0,1,9,4846
8,15,0,52%,394,Sem Info,16,3,0,5,2,4856
...,...,...,...,...,...,...,...,...,...,...,...
590,17,0,44%,363,Sem Info,14,4,0,0,8,5305
592,17,0,59%,552,Sem Info,15,2,0,0,6,5306
595,16,0,57%,541,Sem Info,10,1,0,3,4,5307
596,10,0,51%,315,Sem Info,16,3,0,1,4,5308


In [163]:
df_stats_unificado = pd.merge(mandante_stats_renamed, visitante_stats_renamed, on='partida_id')

# Célula 11: Junção final com a tabela principal
df_final_partidas = pd.merge(df_full_cleaned, df_stats_unificado, on='partida_id')

print("Tabela Mestra final criada com sucesso!")
print(f"Dimensões da tabela final: {df_final_partidas.shape}")

Tabela Mestra final criada com sucesso!
Dimensões da tabela final: (3059, 34)


In [164]:
display(df_final_partidas.head())

,partida_id,rodata,data,hora,mandante,visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,...,visitante_chutes,visitante_chutes_no_alvo,visitante_posse_de_bola,visitante_passes,visitante_precisao_passes,visitante_faltas,visitante_cartao_amarelo,visitante_cartao_vermelho,visitante_impedimentos,visitante_escanteios
0,4741,13,2014-08-03,16:00:00,Chapecoense,Flamengo,C. Rodrigues,V. Luxemburgo da Silva,Chapecoense,Arena Condá,...,9,0,65%,509,Sem Info,13,1,0,1,5
1,4808,20,2014-09-10,19:30:00,Palmeiras,Criciuma,D. Silvestre Júnior,G. Dal Pozzo,Palmeiras,Estádio Municipal Paulo Machado de Carvalho,...,10,0,56%,471,Sem Info,17,3,0,1,6
2,4833,22,2014-09-17,22:00:00,Coritiba,Sao Paulo,M. dos Santos Gonçalves,M. Ramalho,Coritiba,Couto Pereira,...,5,0,57%,481,Sem Info,10,1,0,3,2
3,4846,23,2014-09-21,18:30:00,Gremio,Chapecoense,L. Scolari,J. da Silva,Gremio,Arena do Grêmio,...,14,0,58%,477,Sem Info,14,4,0,2,4
4,4856,24,2014-09-25,20:29:00,Atletico-MG,Santos,L. Culpi,E. Alves Moreira,Atletico-MG,Estádio Raimundo Sampaio,...,13,0,48%,355,Sem Info,18,3,0,6,10


### 💾 Salvando a Tabela Mestra

Arquivo final pronto para análise!  
O dataset consolidado foi salvo em CSV para uso futuro e compartilhamento.

In [165]:
# Tirando linhas com estatísticas incompletas
df_final_partidas = df_final_partidas[
    (df_final_partidas['mandante_precisao_passes'] != 'Sem Info') &
    (df_final_partidas['visitante_precisao_passes'] != 'Sem Info')
]

# Salvando a tabela final em um novo arquivo
df_final_partidas.to_csv('Brasileirao_Dataset/partidas_com_estatisticas_completas.csv', index=False)

print("Arquivo 'partidas_com_estatisticas_completas.csv' salvo com sucesso!")

Arquivo 'partidas_com_estatisticas_completas.csv' salvo com sucesso!


## ⭐ Parte 3: Análise de Fatores de Sucesso no Jogo

🎯 **Objetivo:** Descobrir quais fatores de performance estão mais correlacionados com os resultados das partidas, respondendo perguntas da equipe técnica e buscando novos insights!

---

In [166]:
import numpy as np
import plotly.express as px
import plotly.io as pio


# Definindo um tema padrão mais limpo para os gráficos Plotly
pio.templates.default = "plotly_white"

# Cores padrão para os gráficos
COLOR_RESULTADO = {
    'Vitória': 'green',
    'Derrota': 'red',
    'Empate': 'yellow'
}

In [167]:
# Carregando a tabela mestra que preparamos
try:
    df_analise = pd.read_csv('Brasileirao_Dataset/partidas_com_estatisticas_completas.csv')
    print("Tabela mestra 'partidas_com_estatisticas_completas.csv' carregada com sucesso!")
    print(f"A tabela contém {df_analise.shape[0]} partidas e {df_analise.shape[1]} colunas.")
except FileNotFoundError:
    print("Erro: O arquivo 'partidas_com_estatisticas_completas.csv' não foi encontrado.")
    print("Por favor, certifique-se de que o notebook da Parte 1 e 2 foi executado com sucesso.")
# Verificando as primeiras linhas da tabela mestra
df_analise.head(3)

Tabela mestra 'partidas_com_estatisticas_completas.csv' carregada com sucesso!
A tabela contém 2018 partidas e 34 colunas.


,partida_id,rodata,data,hora,mandante,visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,...,visitante_chutes,visitante_chutes_no_alvo,visitante_posse_de_bola,visitante_passes,visitante_precisao_passes,visitante_faltas,visitante_cartao_amarelo,visitante_cartao_vermelho,visitante_impedimentos,visitante_escanteios
0,6126,1,2018-04-14,16:00:00,Cruzeiro,Gremio,L. Venker de Menezes,Renato Gaúcho,Gremio,Mineirão,...,6,2,60%,649,86%,15,1,1,3,4
1,6127,1,2018-04-14,19:02:00,Vitoria,Flamengo,V. Carmo Mancini,M. Nogueira Barbieri,-,Barradão,...,10,5,34%,286,72%,10,2,1,2,5
2,6128,1,2018-04-14,21:03:00,Santos,Ceara,J. Ribeiro Ventura,M. Oliveira Chamusca,Santos,Estádio Municipal Paulo Machado de Carvalho,...,10,2,46%,472,85%,5,1,0,0,2


### Preparação para Análise: Coluna de Resultado

Criamos a coluna **resultado** para classificar cada partida do ponto de vista do mandante:  
**Vitória**, **Empate** ou **Derrota**.

In [168]:
# Limpando os nomes das colunas e dados de times
df_analise.columns = df_analise.columns.str.lower().str.strip()
for col in ['vencedor', 'mandante', 'visitante']:
    df_analise[col] = df_analise[col].str.strip()

# Criando a coluna 'resultado' para o time mandante
conditions = [
    df_analise['vencedor'] == df_analise['mandante'],
    df_analise['vencedor'] == '-'
]
choices = ['Vitória', 'Empate']
df_analise['resultado'] = np.select(conditions, choices, default='Derrota')

# Função auxiliar para obter o resultado da perspectiva do time de interesse
def get_resultado_perspectiva(row, time_analisado):
    """Interpreta o resultado da partida do ponto de vista do time especificado."""
    if row['mandante'] == time_analisado:
        return row['resultado']
    elif row['visitante'] == time_analisado:
        if row['resultado'] == 'Vitória': return 'Derrota'
        if row['resultado'] == 'Derrota': return 'Vitória'
        return 'Empate'
    return None

print("Coluna 'resultado' e função de perspectiva criadas com sucesso.")
print("\nContagem de resultados do mandante:")
print(df_analise['resultado'].value_counts())


Coluna 'resultado' e função de perspectiva criadas com sucesso.

Contagem de resultados do mandante:
resultado
Vitória    1049
Derrota     553
Empate      416
Name: count, dtype: int64


In [169]:
total_partidas = 1049 + 553 + 416
total_partidas

2018

In [170]:
display(df_analise.head(3))


,partida_id,rodata,data,hora,mandante,visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,...,visitante_chutes_no_alvo,visitante_posse_de_bola,visitante_passes,visitante_precisao_passes,visitante_faltas,visitante_cartao_amarelo,visitante_cartao_vermelho,visitante_impedimentos,visitante_escanteios,resultado
0,6126,1,2018-04-14,16:00:00,Cruzeiro,Gremio,L. Venker de Menezes,Renato Gaúcho,Gremio,Mineirão,...,2,60%,649,86%,15,1,1,3,4,Derrota
1,6127,1,2018-04-14,19:02:00,Vitoria,Flamengo,V. Carmo Mancini,M. Nogueira Barbieri,-,Barradão,...,5,34%,286,72%,10,2,1,2,5,Empate
2,6128,1,2018-04-14,21:03:00,Santos,Ceara,J. Ribeiro Ventura,M. Oliveira Chamusca,Santos,Estádio Municipal Paulo Machado de Carvalho,...,2,46%,472,85%,5,1,0,0,2,Vitória


## ⚽ Posse de Bola: Quem Domina, Ganha?

A posse de bola é um dos indicadores mais tradicionais do futebol moderno. Mas será que ter mais a bola realmente aumenta as chances de vitória no Brasileirão?

Nesta seção, analisamos:

- **Como a posse de bola está distribuída entre mandantes e visitantes.**
- **A relação entre posse de bola e resultado final da partida.**
- **Percentual de vitórias, empates e derrotas para o time com maior posse.**

Visualizamos e interpretamos os dados para responder à pergunta:  
> **Ter mais posse de bola é sinônimo de sucesso? Ou o contra-ataque ainda é rei no futebol brasileiro?**

Acompanhe os gráficos e insights a seguir!

In [171]:
df_posse = df_analise.copy()

# Converte colunas de posse para numérico, tratando erros
for col in ['mandante_posse_de_bola', 'visitante_posse_de_bola']:
    df_posse[col] = pd.to_numeric(df_posse[col].astype(str).str.replace('%', ''), errors='coerce')

# Remove linhas sem dados de posse
df_posse.dropna(subset=['mandante_posse_de_bola', 'visitante_posse_de_bola'], inplace=True)

# Identifica o time com mais posse (None se for empate)
df_posse['time_maior_posse'] = np.select(
    [
        df_posse['mandante_posse_de_bola'] > df_posse['visitante_posse_de_bola'],
        df_posse['visitante_posse_de_bola'] > df_posse['mandante_posse_de_bola']
    ],
    [
        df_posse['mandante'],
        df_posse['visitante']
    ],
    default=None
)

# Filtra apenas jogos onde um time teve mais posse que o outro
df_posse_final = df_posse.dropna(subset=['time_maior_posse']).copy()
df_posse_final['resultado_final'] = df_posse_final.apply(lambda row: get_resultado_perspectiva(row, row['time_maior_posse']), axis=1)

# Calcula o resumo
resumo_posse = df_posse_final['resultado_final'].value_counts(normalize=True).mul(100).reset_index()
resumo_posse.columns = ['resultado', 'percentual']

print("--- Resultado para o time com MAIOR posse de bola ---")
display(resumo_posse.round(2))

# Plota o gráfico
fig = px.pie(resumo_posse, names='resultado', values='percentual', title='Resultados de quem teve MAIOR posse de bola (%)',
             color='resultado', color_discrete_map=COLOR_RESULTADO, hole=.3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()



--- Resultado para o time com MAIOR posse de bola ---


,resultado,percentual
0,Derrota,42.65
1,Vitória,36.74
2,Empate,20.61


**Resultados para quem teve MAIOR posse de bola:**

- 🟥 **Derrota:** 42,7%
- 🟩 **Vitória:** 36,7%
- 🟨 **Empate:** 20,6%

> Ter mais posse de bola não garante a vitória — em mais de 4 a cada 10 jogos, o time com maior posse saiu derrotado!

### A vantagem de jogar em casa é real? 🏠

Jogar em casa sempre foi considerado um fator importante no futebol. Mas será que os dados do Brasileirão confirmam essa crença?  
Abaixo, analisamos a proporção de vitórias, empates e derrotas dos times mandantes ao longo dos anos.


In [172]:
resumo_mandante = df_analise['resultado'].value_counts(normalize=True).mul(100).reset_index()
resumo_mandante.columns = ['resultado', 'percentual']

print("--- Resultados Gerais do Mandante ---")
display(resumo_mandante.round(2))

fig = px.pie(resumo_mandante, names='resultado', values='percentual', title='Resultados do Mandante (%)',
             color='resultado', color_discrete_map=COLOR_RESULTADO, hole=.3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()


--- Resultados Gerais do Mandante ---


,resultado,percentual
0,Vitória,51.98
1,Derrota,27.40
2,Empate,20.61


#### 🏠 **Resultados dos Mandantes**


- 🟥 **Derrota:** 27,4%
- 🟩 **Vitória:** 52,0%
- 🟨 **Empate:** 20,6%

> **Conclusão:**  
> Os times mandantes realmente levam vantagem, vencendo mais da metade dos jogos.  
> Fatores como torcida, familiaridade com o estádio e menor desgaste de viagem podem explicar esse fenômeno.

---

## Quem deu mais chutes, venceu mais?

Será que o time que mais finaliza realmente conquista mais vitórias?  
Abaixo, analisamos todas as partidas do Brasileirão para responder essa pergunta:

- **Comparamos o número total de chutes de cada equipe em cada jogo.**
- **Identificamos quem chutou mais (mandante, visitante ou empate).**
- **Calculamos o percentual de vitórias, empates e derrotas para o time que mais finalizou.**



In [173]:
df_chutes = df_analise.copy()

# Identifica o time com mais chutes (None se for empate)
df_chutes['time_mais_chutes'] = np.select(
    [
        df_chutes['mandante_chutes'] > df_chutes['visitante_chutes'],
        df_chutes['visitante_chutes'] > df_chutes['mandante_chutes']
    ],
    [
        df_chutes['mandante'],
        df_chutes['visitante']
    ],
    default=None
)

# Filtra jogos com contagem de chutes diferente
df_chutes_final = df_chutes.dropna(subset=['time_mais_chutes']).copy()
df_chutes_final['resultado_final'] = df_chutes_final.apply(lambda row: get_resultado_perspectiva(row, row['time_mais_chutes']), axis=1)

# Calcula o resumo
resumo_chutes = df_chutes_final['resultado_final'].value_counts(normalize=True).mul(100).reset_index()
resumo_chutes.columns = ['resultado', 'percentual']

print("--- Resultado para o time que CHUTOU MAIS ---")
display(resumo_chutes.round(2))

fig = px.pie(resumo_chutes, names='resultado', values='percentual', title='Resultados de quem chutou MAIS (%)',
             color='resultado', color_discrete_map=COLOR_RESULTADO, hole=.3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

--- Resultado para o time que CHUTOU MAIS ---


,resultado,percentual
0,Vitória,46.02
1,Derrota,33.08
2,Empate,20.90


**Resultados para quem CHUTOU mais:**

- 🟥 **Derrota:** 33,1%
- 🟩 **Vitória:** 46%
- 🟨 **Empate:** 20,9%

> **Conclusão:**  
> Chutar mais aumenta as chances de vitória, mas não garante o resultado!  
> Em cerca de 1 a cada 3 jogos, o time que mais finalizou acabou derrotado.  
> Ou seja, qualidade das finalizações e eficiência ainda são fundamentais no futebol brasileiro.

### Qualidade é mais importante que quantidade? (Chutes no Alvo)

Será que acertar o alvo faz mais diferença do que simplesmente finalizar mais vezes?  
Aqui, analisamos todas as partidas para responder:

- **Comparamos o número de chutes no alvo de cada equipe em cada jogo.**
- **Identificamos quem acertou mais o gol (mandante, visitante ou empate).**
- **Calculamos o percentual de vitórias, empates e derrotas para o time que mais acertou o alvo.**



In [174]:
df_chutes_alvo = df_analise.copy()

# Identifica o time com mais chutes no alvo (None se for empate)
df_chutes_alvo['time_mais_chutes_alvo'] = np.select(
    [
        df_chutes_alvo['mandante_chutes_no_alvo'] > df_chutes_alvo['visitante_chutes_no_alvo'],
        df_chutes_alvo['visitante_chutes_no_alvo'] > df_chutes_alvo['mandante_chutes_no_alvo']
    ],
    [
        df_chutes_alvo['mandante'],
        df_chutes_alvo['visitante']
    ],
    default=None
)

# Filtra jogos com contagem diferente
df_chutes_alvo_final = df_chutes_alvo.dropna(subset=['time_mais_chutes_alvo']).copy()
df_chutes_alvo_final['resultado_final'] = df_chutes_alvo_final.apply(lambda row: get_resultado_perspectiva(row, row['time_mais_chutes_alvo']), axis=1)

# Calcula o resumo
resumo_chutes_alvo = df_chutes_alvo_final['resultado_final'].value_counts(normalize=True).mul(100).reset_index()
resumo_chutes_alvo.columns = ['resultado', 'percentual']

print("--- Resultado para o time com MAIS CHUTES NO ALVO ---")
display(resumo_chutes_alvo.round(2))

fig = px.pie(resumo_chutes_alvo, names='resultado', values='percentual', title='Resultados de quem teve MAIS CHUTES NO ALVO (%)',
             color='resultado', color_discrete_map=COLOR_RESULTADO, hole=.3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

--- Resultado para o time com MAIS CHUTES NO ALVO ---


,resultado,percentual
0,Vitória,59.17
1,Empate,20.64
2,Derrota,20.19


**Resultados para quem teve MAIS CHUTES no ALVO:**

- 🟥 **Derrota:** 20,2%
- 🟩 **Vitória:** 59,2%
- 🟨 **Empate:** 20,6%
> **Conclusão:**  
> Acertar mais o alvo aumenta significativamente as chances de vitória, mas não garante o resultado.  
> Em quase metade dos jogos, o time que mais acertou o gol não saiu vencedor, mostrando que eficiência é fundamental, mas o futebol ainda reserva surpresas!

## ⚡ Contra-Ataque: A Estratégia dos Menos Possuidores

O contra-ataque é uma das táticas mais clássicas do futebol: ceder a posse de bola ao adversário e apostar em transições rápidas para finalizar mais vezes. Mas será que essa estratégia realmente traz resultados no Brasileirão?

### Como analisamos:

- **Critério:** Selecionamos partidas em que um time teve MENOS posse de bola, mas FINALIZOU MAIS vezes que o adversário.
- **Resultado:** Calculamos o percentual de vitórias, empates e derrotas para o time que adotou esse perfil de jogo.


In [175]:
import duckdb

query_contra_ataque = """
-- Usamos uma Common Table Expression (CTE) para clareza
WITH JOGOS_CONTRA_ATAQUE AS (
    SELECT
        -- Determinamos o resultado do ponto de vista do time que contra-atacou
        CASE
            -- Caso 1: O MANDANTE contra-atacou (menos posse, mais chutes)
            WHEN 
                mandante_posse_de_bola < visitante_posse_de_bola AND 
                mandante_chutes > visitante_chutes
            THEN resultado -- O resultado da partida já é o do mandante

            -- Caso 2: O VISITANTE contra-atacou (menos posse, mais chutes)
            WHEN 
                visitante_posse_de_bola < mandante_posse_de_bola AND 
                visitante_chutes > mandante_chutes
            -- Precisamos inverter o resultado da partida (que é a visão do mandante)
            THEN CASE 
                WHEN resultado = 'Vitória' THEN 'Derrota'
                WHEN resultado = 'Derrota' THEN 'Vitória'
                ELSE 'Empate'
            END
        END AS resultado_final_contra_ataque
    FROM 
        df_analise -- Aqui referenciamos o DataFrame diretamente!
    WHERE
        -- Filtramos apenas as partidas que se encaixam no perfil de contra-ataque
        (mandante_posse_de_bola < visitante_posse_de_bola AND mandante_chutes > visitante_chutes) OR
        (visitante_posse_de_bola < mandante_posse_de_bola AND visitante_chutes > mandante_chutes)
)
-- Agora, agregamos os resultados da CTE
SELECT
    resultado_final_contra_ataque AS resultado,
    COUNT(*) * 100.0 / (SELECT COUNT(*) FROM JOGOS_CONTRA_ATAQUE) AS percentual
FROM
    JOGOS_CONTRA_ATAQUE
GROUP BY
    resultado_final_contra_ataque
ORDER BY
    percentual DESC;
"""

# Executa a consulta e converte o resultado de volta para um DataFrame do pandas
resumo_contra_ataque_sql_df = duckdb.query(query_contra_ataque).to_df()

print("--- Análise (via SQL): Eficácia do Perfil de Contra-Ataque ---\n")
print("Resultado para o time que teve MENOS posse de bola e MAIS chutes:")
display(resumo_contra_ataque_sql_df.round(2))

# Gráfico de pizza dos resultados de quem teve menos posse e mais chutes
fig = px.pie(
    resumo_contra_ataque_sql_df,
    names='resultado',
    values='percentual',
    title='Eficácia do Perfil de Contra-Ataque (%) - Gerado com SQL',
    color='resultado',               # usa o próprio rótulo
    color_discrete_map=COLOR_RESULTADO
)

fig.show()



--- Análise (via SQL): Eficácia do Perfil de Contra-Ataque ---

Resultado para o time que teve MENOS posse de bola e MAIS chutes:


,resultado,percentual
0,Vitória,54.17
1,Derrota,23.85
2,Empate,21.98


**Resultados para quem usou Contra-Ataque:**

- 🟥 **Derrota:** 23,9%
- 🟩 **Vitória:** 54,2%
- 🟨 **Empate:** 22,0%
> **Conclusão:**  
> O contra-ataque é uma estratégia eficiente no Brasileirão!  
> Mais da metade das vezes, o time que teve menos posse e mais chutes saiu vencedor.  
> Isso mostra que, no futebol brasileiro, eficiência e velocidade podem superar o domínio da posse de bola.

---

**Curiosidade:**  
Mesmo sem controlar o jogo, times que apostam em transições rápidas e finalizações conseguem ótimos resultados.  
Aposte na objetividade!